In [1]:
import pandas as pd
import numpy as np

In [2]:
climate_data = pd.read_csv("regional_data.csv")
climate_data.head()

,LAT,LON,YEAR,MO,DY,WS2M,T2M,RH2M,PS,T2MDEW,PRECTOTCORR
0,26.25,73.25,2023,11,1,1.07,28.10,37.71,98.58,11.16,0.0
1,26.25,73.75,2023,11,1,1.30,27.19,41.41,97.94,11.53,0.0
2,26.25,74.25,2023,11,1,1.44,26.11,42.86,97.04,11.19,0.0
3,26.25,74.75,2023,11,1,1.44,25.82,41.78,97.00,10.55,0.0
4,26.25,75.25,2023,11,1,1.40,25.72,41.80,97.43,10.41,0.0


In [3]:
climate_data['WS2M_Change'] = climate_data['WS2M'].diff().fillna(0)
climate_data['T2M_Change'] = climate_data['T2M'].diff().fillna(0)
climate_data['Precip_Change'] = climate_data['PRECTOTCORR'].diff().fillna(0)

In [5]:
climate_data.head()

,LAT,LON,YEAR,MO,DY,WS2M,T2M,RH2M,PS,T2MDEW,PRECTOTCORR,WS2M_Change,T2M_Change,Precip_Change
0,26.25,73.25,2023,11,1,1.07,28.10,37.71,98.58,11.16,0.0,0.00,0.00,0.0
1,26.25,73.75,2023,11,1,1.30,27.19,41.41,97.94,11.53,0.0,0.23,-0.91,0.0
2,26.25,74.25,2023,11,1,1.44,26.11,42.86,97.04,11.19,0.0,0.14,-1.08,0.0
3,26.25,74.75,2023,11,1,1.44,25.82,41.78,97.00,10.55,0.0,0.00,-0.29,0.0
4,26.25,75.25,2023,11,1,1.40,25.72,41.80,97.43,10.41,0.0,-0.04,-0.10,0.0


In [7]:
data_cleaned = climate_data.replace(-999.0, np.nan)

In [8]:
data_cleaned.to_csv("processed_climate_data.csv", index=False)


In [10]:
data = pd.read_csv("processed_climate_data.csv")

ws2m_std = data['WS2M_Change'].std()
t2m_std = data['T2M_Change'].std()
precip_std = data['Precip_Change'].std()

ws2m_mean = data['WS2M_Change'].mean()
t2m_mean = data['T2M_Change'].mean()
precip_mean = data['Precip_Change'].mean()

In [11]:
ws2m_increase_threshold = ws2m_mean + ws2m_std
ws2m_decrease_threshold = ws2m_mean - ws2m_std

t2m_increase_threshold = t2m_mean + t2m_std
t2m_decrease_threshold = t2m_mean - t2m_std

precip_increase_threshold = precip_mean + precip_std
precip_decrease_threshold = precip_mean - precip_std

In [12]:
def classify_event(row):
    if row['WS2M_Change'] > ws2m_increase_threshold:
        return "Wind Increase"
    elif row['WS2M_Change'] < ws2m_decrease_threshold:
        return "Wind Decrease"
    elif row['T2M_Change'] > t2m_increase_threshold:
        return "Temp Increase"
    elif row['T2M_Change'] < t2m_decrease_threshold:
        return "Temp Decrease"
    elif row['Precip_Change'] > precip_increase_threshold:
        return "Rain Predicted"
    elif row['Precip_Change'] < precip_decrease_threshold:
        return "Dry Conditions"
    else:
        return "Normal"

In [13]:
data['Event'] = data.apply(classify_event, axis=1)

In [14]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import to_categorical

In [15]:
features = data[['WS2M_Change', 'T2M_Change', 'Precip_Change']].fillna(0)  
labels = data['Event'].astype('category').cat.codes  
labels_encoded = to_categorical(labels)  

X_train, X_test, y_train, y_test = train_test_split(features, labels_encoded, test_size=0.2, random_state=42)

In [17]:
model = Sequential([
    Input(shape=(X_train.shape[1],)), 
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(labels_encoded.shape[1], activation='softmax')  
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1)

Epoch 1/20
1318/1318 [==============================] - 3s 2ms/step - loss: 0.1459 - accuracy: 0.9887 - val_loss: 0.0132 - val_accuracy: 0.9959
Epoch 2/20
1318/1318 [==============================] - 2s 2ms/step - loss: 0.0171 - accuracy: 0.9963 - val_loss: 0.0080 - val_accuracy: 0.9977
Epoch 3/20
1318/1318 [==============================] - 2s 2ms/step - loss: 0.0090 - accuracy: 0.9970 - val_loss: 0.0100 - val_accuracy: 0.9959
Epoch 4/20
1318/1318 [==============================] - 2s 2ms/step - loss: 0.0072 - accuracy: 0.9974 - val_loss: 0.0056 - val_accuracy: 0.9977
Epoch 5/20
1318/1318 [==============================] - 2s 2ms/step - loss: 0.0079 - accuracy: 0.9974 - val_loss: 0.0049 - val_accuracy: 0.9994
Epoch 6/20
1318/1318 [==============================] - 2s 1ms/step - loss: 0.0061 - accuracy: 0.9979 - val_loss: 0.0057 - val_accuracy: 0.9991
Epoch 7/20
1318/1318 [==============================] - 3s 2ms/step - loss: 0.0066 - accuracy: 0.9975 - val_loss: 0.0070 - val_accuracy:

In [18]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

366/366 [==============================] - 1s 3ms/step - loss: 0.4625 - accuracy: 0.9986
Test Loss: 0.4625018537044525
Test Accuracy: 0.998633861541748


In [19]:
# Save the model to a file
model.save('climate_event_model.h5')
